In [8]:
# Standard libraries
import sys
import os
import json
import importlib
import string
import math
import re
import pprint
# Third party libraries
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

pp = pprint.PrettyPrinter(indent=2, width=100)
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
print('Plotly ver: ', plotly.__version__)
print('Pandas ver: ', pd.__version__)
pd.options.display.max_rows = 150

Plotly ver:  3.2.1
Pandas ver:  0.23.4


## Initialization Function  - read the first sheet of the spreadsheet 
Run in startup before calling functions 

In [9]:
df1 = pd.read_excel("./chronicgdayconsumers.xlsx", sheet_name = 'L1_Consumers_only_g_day', header = 2)
df1.set_index(['Country', 'Pop Class', 'Foodex L1', 'Metrics', 'Survey'])    

Nr Consumers  \
Country Pop Class      Foodex L1                                          Metrics     Survey                                                             
Austria Adults         Grains and grain-based products                    A.01.000001 Austrian Study on Nutritional Status 2010-12 - ...         305.0   
                       Vegetables and vegetable products (including fungi A.01.000317 Austrian Study on Nutritional Status 2010-12 - ...         246.0   
                       Starchy roots and tubers                           A.01.000467 Austrian Study on Nutritional Status 2010-12 - ...         109.0   
                       Legumes, nuts and oilseeds                         A.01.000486 Austrian Study on Nutritional Status 2010-12 - ...          71.0   
                       Fruit and fruit products                           A.01.000544 Austrian Study on Nutritional Status 2010-12 - ...         263.0   
                       Meat and meat products (including edible offal)    A.01.000727 Austrian Study on Nutritional Status 2010-12 - ...         251.0   
                       Fish and other seafood (including amphibians, rept A.01.000876 Austrian Study on Nutritional Status 2010-12 - ...          64.0   
                       Milk and dairy products                            A.01.000948 Austrian Study on Nutritional Status 2010-12 - ...         294.0   
                       Eggs and egg products                              A.01.001252 Austrian Study on Nutritional Status 2010-12 - ...          33.0   
                       Sugar and confectionary                            A.01.001267 Austrian Study on Nutritional Status 2010-12 - ...         223.0   
                       Animal and vegetable fats and oils                 A.01.001346 Austrian Study on Nutritional Status 2010-12 - ...         263.0   
                       Fruit and vegetable juices                         A.01.001394 Austrian Study on Nutritional Status 2010-12 - ...         156.0   
                       Non-alcoholic beverages (excepting milk based beve A.01.001470 Austrian Study on Nutritional Status 2010-12 - ...         287.0   
                       Alcoholic beverages                                A.01.001534 Austrian Study on Nutritional Status 2010-12 - ...         144.0   
                       Drinking water (water without any additives except A.01.001573 Austrian Study on Nutritional Status 2010-12 - ...         300.0   
                       Herbs, spices and condiments                       A.01.001580 Austrian Study on Nutritional Status 2010-12 - ...         270.0   
                       Products for special nutritional use               A.01.001748 Austrian Study on Nutritional Status 2010-12 - ...          11.0   
                       Composite food (including frozen products)         A.01.001789 Austrian Study on Nutritional Status 2010-12 - ...         281.0   
                       Snacks, desserts, and other foods                  A.01.001877 Austrian Study on Nutritional Status 2010-12 - ...          98.0   
        Elderly        Grains and grain-based products                    A.01.000001 Austrian Study on Nutritional Status 2010-12 - ...          67.0   
                       Vegetables and vegetable products (including fungi A.01.000317 Austrian Study on Nutritional Status 2010-12 - ...          47.0   
                       Starchy roots and tubers                           A.01.000467 Austrian Study on Nutritional Status 2010-12 - ...          27.0   
                       Legumes, nuts and oilseeds                         A.01.000486 Austrian Study on Nutritional Status 2010-12 - ...          12.0   
                       Fruit and fruit products                           A.01.000544 Austrian Study on Nutritional Status 2010-12 - ...          62.0   
                       Meat and meat products (including edible offal)    A.01.000727 Austrian Study on Nutritional Status 2010-12 - .

In [60]:
# df1[df1['Country'] == 'Germany'] 
# df1.columns

# df1[~(df1.Metrics == 'A.01.001573') & (df1.Country == 'Austria')]

# df1[df1['Foodex L1'] == 'Drinking water (water without any additives except']

##  `plot_by_L1_and_population()` function

In [10]:
def plot_by_L1_and_population(df, L1_key, population_key, display_mean = False):
    '''
    df            :  L1 dataframe from spreeadsheet
    L1_key        :  L1 Food group to filter on
    population_key:  Population Class to filter on 
    display_mean  :  Display mean trend
    '''
    population_filter = (df['Pop Class'] == population_key)
    L1_filter         = (df['Foodex L1'] == 'Grains and grain-based products')
    stats             = df[L1_filter & population_filter]
    g_stats = stats.groupby('Country', sort = False).max()
#     g_stats

    y_vals = {}
    for country in g_stats.index:
        y_vals[country] = [ g_stats.loc[country][i] for i in ['P5', 'P10','P10','Median','P95','P95','P99']]
#         print(country, ' : ', y_vals[country])
    results = pd.DataFrame(y_vals)
#     results.head(20)

#     print(L1_key)
    data = []
    for col in results.columns:
        data.append(  go.Box( y=results[col], name=col, showlegend=False ) )

    if display_mean:
        data.append( go.Scatter( x = results.columns, y = results.mean(), mode='lines', name='mean' ) )
        
    layout = go.Layout(
        title='Consumption of '+L1_key+' for '+population_key,
        xaxis=dict(
            title='Country',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='grams/day',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
    fig = go.Figure(data=data, layout=layout)

    # IPython notebook
    return py.iplot(fig, filename='pandas-box-plot')
    
    # If want to return in Python"
    # url = py.plot(data, filename='pandas-box-plot')
    

### Function call for `plot_by_L1_and_Population`

In [11]:
country_key = 'Austria'
population_key = 'Adults'
L1_key = 'Grains and grain-based products'

## Call Function 
##--------------------------------------------------------
plot_by_L1_and_population(df1, L1_key, population_key, display_mean = False)

##  `sum_consumption_by_all_countries()` functions

In [4]:
def sum_consumption_by_all_countries_without_water(df):
    gstats= df1[~(df1.Metrics == 'A.01.001573')].groupby(['Country', 'Survey', 'Pop Class'], sort = False)['Mean'].sum().groupby(['Country', 'Pop Class'], sort = False).mean()
    return gstats.round(0)

def sum_consumption_by_all_countries_water(df):
    gstats= df1[(df1.Metrics == 'A.01.001573')].groupby(['Country', 'Survey', 'Pop Class'], sort = False)['Mean'].sum().groupby(['Country', 'Pop Class'], sort = False).mean()
    return gstats.round(0)

### Function call for `sum_consumption_by_all_countries_without_water()`

In [54]:
## Call Function 
##--------------------------------------------------------
answer = sum_consumption_by_all_countries_without_water(df1)
print(answer)

Country         Pop Class      
Austria         Adults             2664.0
                Elderly            2268.0
                Very elderly       2431.0
                Other children     1833.0
                Adolescents        1533.0
Belgium         Adolescents        2787.0
                Adults             2828.0
                Elderly            2601.0
                Very elderly       2444.0
                Toddlers           1627.0
                Other children     1611.0
Bulgaria        Infants             909.0
                Toddlers           1252.0
                Other children     1437.0
Cyprus          Adolescents        1657.0
Czech Republic  Other children     1807.0
                Adolescents        2385.0
                Adults             2933.0
Germany         Infants            1088.0
                Toddlers           1186.0
                Other children     1536.0
                Adolescents        2317.0
                Adults             3340.0
  

### Function call for `sum_consumption_by_all_countries_water()`

In [56]:
## Call Function 
##--------------------------------------------------------
answer = sum_consumption_by_all_countries_water(df1)
print(answer)

Country         Pop Class      
Austria         Adults             1175.0
                Elderly             789.0
                Very elderly       1114.0
                Other children      577.0
                Adolescents         555.0
Belgium         Adolescents         700.0
                Adults              786.0
                Elderly             641.0
                Very elderly        531.0
                Toddlers            172.0
                Other children      256.0
Bulgaria        Infants             349.0
                Toddlers            373.0
                Other children      467.0
Cyprus          Adolescents          83.0
Czech Republic  Other children      614.0
                Adolescents         852.0
                Adults             1313.0
Germany         Infants             176.0
                Toddlers            324.0
                Other children      383.0
                Adolescents         725.0
                Adults             1126.0
  

### Example of retrieving information for one country

In [12]:
print(answer['Austria'].to_dict())
# print(answer['Toddlers'])

{'Adolescents': 2089.0, 'Very elderly': 3545.0, 'Elderly': 3058.0, 'Adults': 3839.0, 'Other children': 2410.0}


## Misc Code

In [24]:
df1[(df1['Country']=='Netherlands') & (df1['Pop Class']=='Adults')].grou()

Country         NetherlandsNetherlandsNetherlandsNetherlandsNe...
Survey          Dutch National Dietary Survey 2003Dutch Nation...
Pop Class       AdultsAdultsAdultsAdultsAdultsAdultsAdultsAdul...
Foodex L1       Grains and grain-based productsVegetables and ...
Metrics         A.01.000001A.01.000317A.01.000467A.01.000486A....
Nr Consumers                                                37377
% Consumers                                               26.4307
Mean                                                      8408.52
STD                                                       6737.92
P5                                                         1243.5
P10                                                       2052.42
Median                                                    6799.41
P95                                                       21881.2
P97.5                                                     25620.3
P99                                                       30968.2
Comment   

In [27]:
    g_stats = df1.groupby(['Country', 'Survey', 'Pop Class', 'Foodex L1'], sort = False)['Mean'].sum()

g_stats

In [ ]:
def sum_consumption_by_country(df, country_key):
    country_filter = (df['Country'] == country_key)
    g_stats = df[country_filter].groupby(['Country', 'Survey', 'Pop Class'], sort = False)['Mean'].sum().groupby('Pop Class', sort = False).mean().sort_index()
    return g_stats.round(0)


In [119]:
## Call Function 
##--------------------------------------------------------
answer = sum_consumption_by_country(df1, 'Denmark')
print(answer.to_dict())
print(answer['Toddlers'])

{'Adults': 3745.0, 'Toddlers': 1482.0, 'Infants': 1253.0, 'Very elderly': 3516.0, 'Adolescents': 2670.0, 'Elderly': 3512.0, 'Other children': 2176.0}
1482.0


In [ ]:
#df2.head()
#df3.head()
#df4.head()
print(df1.columns)
L1_foodgroups = df1['Foodex L1' ].unique()
print('L1 Food Groups ', len(L1_foodgroups))
pp.pprint(L1_foodgroups)
print(df1['Survey'].unique())

In [79]:
# df1[ (df1['Country'] == 'Denmark') & (df1['Pop Class'] =='Very elderly') ]

In [86]:
pd.options.display.max_rows = 100
# df1[ (df1['Country'] == 'Denmark') & (df1['Survey'] == 'Danish Dietary Survey')][['Pop Class','Nr Consumers', 'Foodex L1']]      ##.groupby('Survey').max()
g_stats = df1.groupby(['Country', 'Survey', 'Pop Class'], sort = False)['Mean'].sum().groupby(['Country', 'Pop Class'], sort = False).mean()
g_stats

Country         Pop Class      
Austria         Adults             3839.437782
                Elderly            3057.579458
                Very elderly       3544.986866
                Other children     2409.847723
                Adolescents        2088.809800
Belgium         Adolescents        3486.831708
                Adults             3613.969194
                Elderly            3241.531193
                Very elderly       2975.445205
                Toddlers           1798.948146
                Other children     1867.366346
Bulgaria        Infants            1257.148264
                Toddlers           1625.762263
                Other children     1904.010407
Cyprus          Adolescents        1740.196706
Czech Republic  Other children     2420.936852
                Adolescents        3237.228492
                Adults             4245.855166
Germany         Infants            1264.790288
                Toddlers           1510.009145
                Other childr

In [37]:
answer.groupby('Pop Class', sort = False).mean()

,Nr Consumers,% Consumers,Mean,STD,P5,P10,Median,P95,P97.5,P99
Pop Class,,,,,,,,,,
Other children,6048.0,15.376962,2175.527104,1296.847200,591.170571,808.258643,1918.410464,4620.230071,5286.549929,6479.363000
Adolescents,6604.0,15.437793,2670.357110,1946.447901,528.753726,768.606071,2201.432393,6349.908929,7657.122464,9999.237714
Adults,36479.0,16.041058,3745.038567,2668.616688,628.394405,985.688857,3194.205179,8620.107857,10348.450143,12585.850143
Elderly,4667.5,16.032823,3511.660884,2374.173320,709.479214,1044.611571,2992.090964,7783.178786,9593.492429,11106.734429
Very elderly,250.5,15.741667,3516.182687,2162.502378,870.488143,1178.561071,3146.006036,7613.816786,8328.061786,8328.061786
Infants,10652.0,12.895884,1252.926600,992.123960,124.130143,233.431571,1038.443857,3249.442000,3684.190714,4441.557857
Toddlers,13680.0,14.918212,1481.534565,863.029800,447.648714,616.361571,1306.977286,3016.441000,3495.953857,4195.454714
